# Export tick updates from multiple assets using dedicated service - Python

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument on multiple data sources`. 

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickUpdatesService*: to directly retrieve tick updates from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    
***

# Run tick updates multi asset extraction sample

### Step 1: Install packages

In [1]:
pip install googleapis-common-protos protobuf grpcio pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install systemathics.apis --pre

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import google.protobuf.wrappers_pb2 as wrappers
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.identifier_and_level_pb2 as identifier_and_level
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_updates_pb2 as tick_updates
import systemathics.apis.services.tick.v1.tick_updates_pb2_grpc as tick_updates_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [4]:
token = token_helpers.get_token()
display(token)

'Bearer eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlpBV05FbDJfMF9zTDNUTklIdlN3SiJ9.eyJodHRwOi8vc2NoZW1hcy54bWxzb2FwLm9yZy93cy8yMDA1LzA1L2lkZW50aXR5L2NsYWltcy9lbWFpbGFkZHJlc3MiOiJvY3RhdmUuYmVsb3RAc3lzdGVtYXRoaWNzLmNvbSIsImlzcyI6Imh0dHBzOi8vZ2FueW1lZGUtZGV2LmV1LmF1dGgwLmNvbS8iLCJzdWIiOiJhdXRoMHw2MjRmZWE2NWJlODIzYzAwNmZlMzg2MzQiLCJhdWQiOlsiaHR0cHM6Ly9kZXYuZ2FueW1lZGUtZGV2IiwiaHR0cHM6Ly9nYW55bWVkZS1kZXYuZXUuYXV0aDAuY29tL3VzZXJpbmZvIl0sImlhdCI6MTY0OTQwNjA3NiwiZXhwIjoxNjUxOTk4MDc2LCJhenAiOiJubGp0SkhkRnJ1Z3VDY2ZacUVtZHEyMEtrMkM2dFduWSIsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZW1haWwiLCJwZXJtaXNzaW9ucyI6WyJhZG1pbiIsImFydGVmYWN0cyIsImNoYW5nZXMiLCJjb21wb25lbnRzIiwiZGFpbHlfYmFycyIsImRhaWx5X2JvbGxpbmdlciIsImRhaWx5X2NtYSIsImRhaWx5X2VtYSIsImRhaWx5X21hY2QiLCJkYWlseV9wcmljZXMiLCJkYWlseV9yc2kiLCJkYWlseV9zbWEiLCJkYWlseV92b2xhdGlsaXR5IiwiZGFpbHlfdndhcHMiLCJkaXZpZGVuZHMiLCJpZGVudGlmaWVyX21hcHBpbmciLCJpbnRyYWRheV9iYXJzIiwiaW50cmFkYXlfYm9sbGluZ2VyIiwiaW50cmFkYXlfY21hIiwiaW50cmFkYXlfZW1hIiwiaW50cmFkYXlfbWFjZCIsIml

### Step 3: Create and process request
To request *tick updates* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

#### 3.1 Instrument selection

In [5]:
# Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
# The ICE ticker
ticker = 'E:BNP'

#### 3.2 Time period delimitation
The following code setups time constraints

In [6]:
# Create date intervals (we are using Google date format)
# Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day = 1), 
    end_date = date.Date(year = 2021, month = 11, day = 1)
)

# Build the tick quotes request time interval (we are using Google date time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 20, minutes = 30, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [7]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

The following code snippets create the market data request and instantiate the service:

In [8]:
# We want to stream the ticks from those two Euronext sources (specific to provider "ICE")
my_provider = 'ICE'
identifier_1 = identifier_and_level.IdentifierAndLevel(ticker = ticker,exchange = '787',level = level.LEVEL_TRADES, provider = wrappers.StringValue(value=my_provider)  )
identifier_2 = identifier_and_level.IdentifierAndLevel(ticker = ticker,exchange = 'EQUITY_L2_973', level = level.LEVEL_TRADES_AND_BOOK, provider = wrappers.StringValue(value=my_provider) )
my_identifiers = [identifier_1, identifier_2]

In [9]:
# generate the tick updates request
request = tick_updates.TickUpdatesRequest(
    identifiers = my_identifiers,
    constraints = my_constraints,
    adjustment = False
)

#### 3.4 Process request and export results

We define beforehand some functions to get a better display

In [10]:
# Get side from code
def get_side(side):
    if side == 0:
        return 'SIDE_UNSPECIFIED'
    elif side ==1:
        return 'SIDE_BID'
    elif side ==1:
        return 'SIDE_ASK'
    else:
        return ''

# Get action from code
def get_action(action):
    if action == 0:
        return 'ACTION_UNSPECIFIED'
    elif action ==1:
        return 'ACTION_SET'
    elif action ==2:
        return 'ACTION_CLEAR'
    else:
        return ''
    
# Get field name type from code
def get_field_name(field):
    if field == 9:
        return 'FIELD_TRADE_PRICE'
    elif field ==10:
        return 'FIELD_TRADE_SIZE'
    elif field ==11:
        return 'FIELD_TRADING_CONDITION'
    elif field ==25:
        return 'FIELD_TRADE_ID'
    else:
        return ''
    
# Get field type type from code
def get_field_type(field):
    if field == 9:
        return 'double_value'
    elif field ==10:
        return 'long_value'
    elif field ==11 or field == 25:
        return 'string_value'
    else:
        return ''
    
# Get field value type from update
def get_field_value(update):
    my_type = get_field_type(update.field)
    if my_type == 'double_value':
        return update.double_value
    elif my_type == 'long_value':
        return update.long_value
    elif my_type == 'string_value':
        return update.string_value
    else:
        return ''

In the following code snippet, we request tick updates for the given instruments.
The streamed response is exported on the fly in a file.

In [11]:
import csv
from datetime import timedelta

filename = '{}_tick_updates.csv'.format(ticker)
total_messages,mbo_count,mbl_count,fields_count,mappings_count = 0,0,0,0,0

with open(filename, mode='w') as tick_uptades_file:
    tick_uptades_writer = csv.writer(tick_uptades_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write 1rst row
    tick_uptades_writer.writerow(['Timestamp', 'Type', 'Mapping', 'Action', 'Side', 'Price', 'Size', 'Depth', 'OrderId', 'Condition', 'Reset', 'FieldName', 'FieldType', 'FieldValue', 'MessageNumber'])
    try:
        # open a gRPC channel
        with channel_helpers.get_grpc_channel() as channel:

            # instantiate the tick raw service
            service = tick_updates_service.TickUpdatesServiceStub(channel)

            # process the tick raw request
            for current in service.TickUpdates(request=request, metadata=[('authorization', token)]):
                
                total_messages+=1
                if len(current.mapping.values) > 0: # mapping
                    my_mapping = current.mapping
                    mappings_count +=1
                    
                if current.fields_updates.time_stamp.seconds > 0: # fields update
                    test_fieldupdate = current
                    fields_count+=1
                    
                    str_timestamp = datetime.fromtimestamp(current.fields_updates.time_stamp.seconds) + timedelta(microseconds = current.fields_updates.time_stamp.nanos/1000)
                    str_type = 'FLD'
                    str_mapping = current.fields_updates.mapping
                    str_reset =current.fields_updates.reset
                    for update in current.fields_updates.updates:
                        str_action =''
                        str_side =''
                        str_price =''
                        str_size =''
                        str_depth =''
                        str_orderid =''
                        str_condition =''
                        str_fieldname =get_field_name(update.field)
                        str_fieldtype =get_field_type(update.field)
                        str_fieldvalue =get_field_value(update)
                        str_messagenumber = total_messages
                        tick_uptades_writer.writerow([str_timestamp, str_type, str_mapping, str_action, str_side, str_price, str_size, str_depth, str_orderid, str_condition, str_reset, str_fieldname, str_fieldtype, str_fieldvalue, str_messagenumber])

                    
                if current.mbl_book_updates.time_stamp.seconds > 0: # mbl update
                    mbl_count+=1
                    test_mbl = current
                    
                    str_timestamp = datetime.fromtimestamp(current.mbl_book_updates.time_stamp.seconds) + timedelta(microseconds = current.mbl_book_updates.time_stamp.nanos/1000)
                    str_type = 'MBL'
                    str_mapping = current.mbl_book_updates.mapping
                    str_action =get_action(current.mbl_book_updates.updates[0].action)
                    str_side =get_side(current.mbl_book_updates.updates[0].side)
                    str_price =current.mbl_book_updates.updates[0].price.value
                    str_size =current.mbl_book_updates.updates[0].size.value
                    str_depth =current.mbl_book_updates.updates[0].depth
                    str_orderid =''
                    str_condition =''
                    str_reset =current.mbl_book_updates.reset
                    str_fieldname =''
                    str_fieldtype =''
                    str_fieldvalue =''
                    str_messagenumber = total_messages
                    tick_uptades_writer.writerow([str_timestamp, str_type, str_mapping, str_action, str_side, str_price, str_size, str_depth, str_orderid, str_condition, str_reset, str_fieldname, str_fieldtype, str_fieldvalue, str_messagenumber])
                                    
                if current.mbo_book_updates.time_stamp.seconds > 0: # mbo update
                    mbo_count+=1
                    test_mbo = current
                    
                    str_timestamp = datetime.fromtimestamp(current.mbo_book_updates.time_stamp.seconds) + timedelta(microseconds = current.mbo_book_updates.time_stamp.nanos/1000)
                    str_type = 'MBO'
                    str_mapping = current.mbo_book_updates.mapping
                    str_action =get_action(current.mbo_book_updates.updates[0].action)
                    str_side =get_side(current.mbo_book_updates.updates[0].side)
                    str_price =current.mbo_book_updates.updates[0].price.value
                    str_size =current.mbo_book_updates.updates[0].size.value
                    str_depth =''
                    str_orderid =current.mbo_book_updates.updates[0].order_id
                    str_condition =current.mbo_book_updates.updates[0].condition
                    str_reset =current.mbo_book_updates.reset
                    str_fieldname =''
                    str_fieldtype =''
                    str_fieldvalue =''
                    str_messagenumber = total_messages
                    tick_uptades_writer.writerow([str_timestamp, str_type, str_mapping, str_action, str_side, str_price, str_size, str_depth, str_orderid, str_condition, str_reset, str_fieldname, str_fieldtype, str_fieldvalue, str_messagenumber])
                                  
    except grpc.RpcError as e:
        display(e.code().name)
        display(e.details())

# Some display
print('----- Done exporting ----- ')
print('Updates found: {}'.format(total_messages)) 
print('--> Mappings found: {}'.format(mappings_count)) 
print('--> MBO updates found: {}'.format(mbo_count)) 
print('--> MBL updates found: {}'.format(mbl_count)) 
print('--> Fields updates found: {}'.format(fields_count)) 
print('Check export in {}'.format(filename)) 

----- Done exporting ----- 
Updates found: 269772
--> Mappings found: 1
--> MBO updates found: 258387
--> MBL updates found: 0
--> Fields updates found: 11384
Check export in E:BNP_tick_updates.csv
